In [1]:
import numpy as np

import sensors
import em_stoch

In [2]:
DIST_RATIO = 0.5

In [3]:
Num_sensors1 = 5
Num_emitters1 = 1
sample_size1 = 100
theta1_rad = [0.7] # Угловые координаты источников (DoA) в радианах
theta1_deg = np.rad2deg(theta1_rad) # Угловые координаты источников (DoA) в градусах
P_1 = 1 * np.eye(Num_emitters1, dtype=np.float64) # Ковариация сигналов
Q_1 = 1.1 * np.eye(Num_sensors1, dtype=np.float64) # Ковариация шумов
A1 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors1).reshape(-1,1) * 
             np.sin(theta1_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S1 = sensors.gss(Num_emitters1, sample_size1, P_1)
N1 = sensors.gss(Num_sensors1, sample_size1, Q_1)
X1 = (A1 @ S1.T + N1.T).T

Num_sensors2 = 5
Num_emitters2 = 2
sample_size2 = 100
theta2_rad = np.array([-0.5, 0.7], dtype=np.float64) # Угловые координаты источников (DoA) в радианах
theta2_deg = np.rad2deg(theta2_rad) # Угловые координаты источников (DoA) в градусах
P_2 = 1 * np.eye(Num_emitters2, dtype=np.float64) # Ковариация сигналов
Q_2 = 2.1 * np.eye(Num_sensors2, dtype=np.float64) # Ковариация шумов
A2 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors2).reshape(-1,1) * 
             np.sin(theta2_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S2 = sensors.gss(Num_emitters2, sample_size2, P_2)
N2 = sensors.gss(Num_sensors2, sample_size2, Q_2)
X2 = (A2 @ S2.T + N2.T).T

In [4]:
X1_with_mv = sensors.MCAR(X1, [2,4], [0.4, 0.4])
X2_with_mv = sensors.MCAR(X2, [2,4], [0.4, 0.4])

In [5]:
X1_with_mv

array([[ 1.23286261e+00+0.9418559j , -1.85659807e-02-2.84696988j,
                    nan+0.j        ,  2.32727443e-01-0.74680402j,
                    nan+0.j        ],
       [-1.84555688e+00-0.03615284j,  5.79029362e-01+0.98613426j,
         2.52068362e-01-0.41735757j, -8.97514167e-01+0.04994758j,
                    nan+0.j        ],
       [-1.09664728e+00+1.1646421j , -1.72871621e-01-0.7934921j ,
         1.76743201e+00-1.60581369j, -1.22187416e+00+0.6652129j ,
                    nan+0.j        ],
       [-3.19705011e+00-1.03130086j,  6.19777131e-01+1.91974302j,
         2.07216841e+00-3.48689659j, -2.50062242e+00-0.98218031j,
        -3.00018978e-01+1.93779552j],
       [-1.05663395e+00-0.81800175j, -7.86290658e-01+1.43634167j,
                    nan+0.j        , -3.21607249e-01-0.98971863j,
         3.63068447e-01+1.76799945j],
       [-4.64436551e-01+1.06533348j,  4.39830960e-01+1.36970282j,
                    nan+0.j        , -1.05030240e+00+0.27738488j,
                  

In [6]:
print(np.trace(sensors.initial_Cov(X1_with_mv)))
print(np.trace(sensors.complex_cov(X1)))

(10.280945744938716+0j)
(11.105494677649398+0j)


In [7]:
EM_theta1_rad, EM_P1, lhd_1 = em_stoch.multi_start_EM(X1, Num_emitters1, Q_1, 10, 
                                                         max_iter=20, 
                                                         rtol=1e-3)

0-th start
theta=[0.15335215],P=[0.53943744]
1-th start
theta=[0.13636065],P=[0.64801961]
2-th start
theta=[1.40627821],P=[0.44565758]
3-th start
theta=[-0.15355298],P=[0.001]
4-th start
theta=[0.5300775],P=[7.73083076]
5-th start
theta=[0.60846218],P=[10.1946531]
6-th start
theta=[-1.46911153],P=[0.26942379]
7-th start
theta=[-1.07899142],P=[0.25455097]
8-th start
theta=[-1.31513736],P=[0.13317456]
9-th start
theta=[-0.20985359],P=[0.001]
best_start=5


In [8]:
print(EM_theta1_rad, EM_P1)

[0.60846218] [[10.1946531]]


In [9]:
EM_theta1_deg = np.rad2deg(EM_theta1_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(EM_theta1_deg-theta1_deg)},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta1_rad)-np.sin(theta1_rad))}.")

Погрешность в смысле разности углов в градусах:       [5.2447308],
Погрешность в смысле разности синусов углов:       [0.07261138].


In [10]:
EM_theta2_rad, EM_P2, lhd_2 = em_stoch.multi_start_EM(X2_with_mv, Num_emitters2, Q_2, 10, 
                                                         max_iter=20, 
                                                         rtol=1e-3)

0-th start
theta=[-1.41744418  0.15335215],P=[1.00000000e-03 1.29011409e+00]
likelihood is -1133.480465604101 on iteration 0
likelihood is -1130.9909165274678 on iteration 1
likelihood is -1129.2053241272024 on iteration 2
likelihood is -1127.954870624665 on iteration 3
likelihood is -1127.0669621984177 on iteration 4
likelihood is -1126.4274459919682 on iteration 5
likelihood is -1125.9609022900254 on iteration 6
likelihood is -1125.6171814296722 on iteration 7
likelihood is -1125.3623322226235 on iteration 8
likelihood is -1125.1728978620952 on iteration 9
likelihood is -1125.032351194726 on iteration 10
likelihood is -1124.9288340942214 on iteration 11
likelihood is -1124.8536922043568 on iteration 12
likelihood is -1124.800513756172 on iteration 13
likelihood is -1124.7644779093316 on iteration 14
likelihood is -1124.7419336921982 on iteration 15
likelihood is -1124.7300635674426 on iteration 16
likelihood is -1124.7266820889593 on iteration 17
likelihood is -1124.7300715344734 on 

In [11]:
print(EM_theta2_rad, EM_P2)

[0.20376006 0.22937217] [[ 0.00099832  0.        ]
 [ 0.         -0.71323164]]


In [12]:
EM_theta2_deg = np.rad2deg(EM_theta2_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(np.sort(EM_theta2_deg)-np.sort(theta2_deg))},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta2_rad)-np.sin(theta2_rad))}.")

Погрешность в смысле разности углов в градусах:       [40.32248136 26.9649882 ],
Погрешность в смысле разности синусов углов:       [0.68177857 0.4168515 ].


In [13]:
em_stoch.incomplete_lkhd(X1, theta1_rad, P_1, Q_1)

np.float64(-731.4574707297775)

In [14]:
em_stoch.incomplete_lkhd(X2, theta2_rad, P_2, Q_2)

np.float64(-1121.389306902635)